<a href="https://colab.research.google.com/github/tharunj-droid/100-days-of-code/blob/main/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The code imports necessary libraries for data analysis and text analysis.
The TfidfVectorizer function is used to convert raw text documents into a matrix of TF-IDF features.
The linear_kernel function is used to compute cosine similarities between vectors of these features.
The pairwise_distances function is used to calculate pairwise distances between the vectors.
This code is likely being used to find similar documents or articles based on their content.
It's important to understand the context and goals of the larger data analysis project in order to fully understand what this code is doing.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
movies_df = pd.read_csv("/content/movies.csv")
rating_df = pd.read_csv("/content/ratings.csv")

movies_df.head() is a method used to display the first few rows of a dataframe called movies_df. Specifically, it displays the first 5 rows by default unless a different number is specified in the parentheses.

In [ ]:
movies_df.head()

this code creates a bar chart showing the most popular genres of movies released. It does this by creating a list of genres for each movie, counting the number of movies in each genre, and creating a bar chart based on the genre counts.

In [ ]:
#Most popular genres of movie released
plt.figure(figsize=(20,7))
generlist = movies_df['genres'].apply(lambda generlist_movie : str(generlist_movie).split("|"))
geners_count = {}

for generlist_movie in generlist:
    for gener in generlist_movie:
        if(geners_count.get(gener,False)):
            geners_count[gener]=geners_count[gener]+1
        else:
            geners_count[gener] = 1       
geners_count.pop("(no genres listed)")
plt.bar(geners_count.keys(),geners_count.values())

this code creates a histogram of the distribution of user ratings for movies, showing how often each rating value occurs in the dataset.

In [ ]:
#Distribution of users rating
sns.distplot(rating_df["rating"])

 this code prints out the number of rows and columns in the rating_df and movies_df dataframes to the console, with the output separated by a new line and labeled to indicate which dataframe each set of values corresponds to.

In [ ]:
print("Shape of frames: \n"+ " Rating DataFrame"+ str(rating_df.shape)+"\n Movies DataFrame"+ str(movies_df.shape))

this code merges the movies_df and rating_df dataframes into a new dataframe called merge_ratings_movies, using an inner join to match rows based on the "movieId" column. The resulting dataframe contains all columns from both input dataframes.

In [ ]:
merge_ratings_movies = pd.merge(movies_df, rating_df, on='movieId', how='inner')

code displays the first two rows of the merge_ratings_movies dataframe that was created by merging the movies_df and rating_df dataframes based on the "movieId" column.

In [ ]:
merge_ratings_movies.head(2)

 this code drops the "timestamp" column from the merge_ratings_movies dataframe and prints the resulting shape of the dataframe to show how many rows and columns are left.

In [ ]:
merge_ratings_movies = merge_ratings_movies.drop('timestamp', axis=1)
merge_ratings_movies.shape

code groups the merge_ratings_movies dataframe by "userId" and calculates the size and mean of each group for each column. The resulting ratings_grouped_by_users dataframe displays the size and mean of each group for each column, including "movieId" and "rating"

In [ ]:
#Grouping the rating based on user
ratings_grouped_by_users = merge_ratings_movies.groupby('userId').agg([np.size, np.mean])
ratings_grouped_by_users.head(2)

this code drops the "movieId" column from the ratings_grouped_by_users dataframe and saves the result back to the same variable. The resulting ratings_grouped_by_users dataframe displays the size and mean of each group for only the "rating" column.

In [ ]:
ratings_grouped_by_users = ratings_grouped_by_users.drop('movieId', axis = 1)

, this code groups the merge_ratings_movies dataframe by "movieId" and calculates the mean and size of each group for each column. The resulting ratings_grouped_by_movies dataframe displays the mean and size of each group for each column, including "userId" and "rating". The "userId" column is then removed from the ratings_grouped_by_movies dataframe.

In [ ]:
ratings_grouped_by_movies = merge_ratings_movies.groupby('movieId').agg([np.mean], np.size)
ratings_grouped_by_movies.shape
ratings_grouped_by_movies.head(3)
ratings_grouped_by_movies = ratings_grouped_by_movies.drop('userId', axis=1)

, this code generates a horizontal bar chart that displays the top 10 movies with the highest average rating. The chart shows the movie titles on the y-axis and the average rating on the x-axis.

In [ ]:
#Movies with high average rating
ratings_grouped_by_movies['rating']['mean'].sort_values(ascending=False).head(10).plot(kind='barh', figsize=(7,6));

this code filters and selects the movies with an average rating below 1.5, and then generates a horizontal bar chart displaying the top 20 movies with the lowest average rating. The chart shows the movie titles on the y-axis and the average rating on the x-axis. The code also displays the first 5 rows of the low_rated_movies dataframe, which contains the movie titles and their corresponding mean rating.

In [ ]:
#Movies with low average rating
low_rated_movies_filter = ratings_grouped_by_movies['rating']['mean']< 1.5
low_rated_movies = ratings_grouped_by_movies[low_rated_movies_filter]
low_rated_movies.head(20).plot(kind='barh', figsize=(7,5))
low_rated_movies.head(5)

Content-based filtering

this code creates a matrix of cosine similarity scores between the genres of the movies using the TfidfVectorizer and linear_kernel functions from the scikit-learn library. The resulting cosine_sim_movies matrix contains the cosine similarity scores between each pair of movies based on their genres.

In [ ]:
tfidf_movies_genres = TfidfVectorizer(token_pattern = '[a-zA-Z0-9\-]+')

movies_df['genres'] = movies_df['genres'].replace(to_replace="(no genres listed)", value="")

tfidf_movies_genres_matrix = tfidf_movies_genres.fit_transform(movies_df['genres'])

cosine_sim_movies = linear_kernel(tfidf_movies_genres_matrix, tfidf_movies_genres_matrix)

The function get_recommendations_based_on_genres() takes in a movie title and returns two recommended movies based on their genre similarity, using a pre-calculated cosine similarity matrix. The function first identifies the index of the movie, then uses the cosine similarity scores to find the two movies with the highest similarity scores that are not the same as the input movie. The function has potential limitations in that it only considers genre similarity, may not provide enough recommendations, and relies on a pre-calculated matrix that could become outdated. To improve the function, additional factors could be considered, more recommendations could be provided, and dynamic calculations could be used instead of pre-calculated matrices.

In [ ]:
def get_recommendations_based_on_genres(movie_title, cosine_sim_movies=cosine_sim_movies):
    idx_movie = movies_df.loc[movies_df['title'].isin([movie_title])]
    idx_movie = idx_movie.index
    
    sim_scores_movies = list(enumerate(cosine_sim_movies[idx_movie][0]))
    
    sim_scores_movies = sorted(sim_scores_movies, key=lambda x: x[1], reverse=True)

    sim_scores_movies = sim_scores_movies[1:3]

    movie_indices = [i[0] for i in sim_scores_movies]
    
    return movies_df['title'].iloc[movie_indices]

 the get_recommendations_based_on_genres() function takes a movie title as input, computes the cosine similarity scores between this movie and all other movies based on their genres using the cosine_sim_movies matrix, and returns a list of top recommended movies based on the similarity scores

In [ ]:
get_recommendations_based_on_genres("Father of the Bride Part II (1995)")

The function get_recommendation_content_model takes in a user ID as input and returns a set of recommended movies based on the content of the movies the user has already rated.

First, the function filters the rating dataframe to only include the rows for the specified user ID. Then, it creates a list of all the movies that the user has already rated by iterating over the filtered rating dataframe and extracting the corresponding movie titles from the movies dataframe.

Next, the function loops through each movie in the user's rated movie list and calls the get_recommendations_based_on_genres function to get a list of recommended movies based on the genres of the rated movie. These recommended movies are added to a separate list called recommended_movie_list.

Finally, the function removes any recommended movies that the user has already rated and returns the set of remaining recommended movies

In [ ]:
def get_recommendation_content_model(userId):
    recommended_movie_list = []
    movie_list = []
    df_rating_filtered = rating_df[rating_df["userId"]== userId]
    for key, row in df_rating_filtered.iterrows():
        movie_list.append((movies_df["title"][row["movieId"]==movies_df["movieId"]]).values) 
    for index, movie in enumerate(movie_list):
        for key, movie_recommended in get_recommendations_based_on_genres(movie[0]).iteritems():
            recommended_movie_list.append(movie_recommended)
            
    for movie_title in recommended_movie_list:
        if movie_title in movie_list:
            recommended_movie_list.remove(movie_title)
    
    return set(recommended_movie_list)
get_recommendation_content_model(1)

The `get_movie_label` function takes a movie id as input and uses K-Nearest Neighbors algorithm to find the cluster label to which the movie belongs based on its genre. It does this by training a classifier using the TF-IDF matrix of the movie genres and the corresponding genre labels from the movies dataframe. The `n_neighbors` parameter of the `KNeighborsClassifier` is set to 5. Once the classifier is trained, it is used to predict the genre label of the movie represented by the input `movie_id`. The predicted label is returned as output.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  
def get_movie_label(movie_id):
    """
    Get the cluster label to which movie belongs by KNN algorithm.  
    :param movie_id: movie id
    :return: genres label to movie belong
    """
    classifier = KNeighborsClassifier(n_neighbors=5)
    x= tfidf_movies_genres_matrix
    y = movies_df.iloc[:,-1]
    classifier.fit(x, y)
    y_pred = classifier.predict(tfidf_movies_genres_matrix[movie_id])
    return y_pred

This code defines a function `evaluate_content_based_model()` which is used to evaluate the performance of the content-based movie recommendation system based on movie genres. It iterates over each movie in the movies dataframe and generates a list of recommended movies by calling the `get_recommendations_based_on_genres()` function for each movie. It then predicts the genre label for each recommended movie by calling the `get_movie_label()` function and compares it with the actual genre label of the movie. The number of correct and incorrect predictions are accumulated and the hit rate and fault rate are computed and printed at the end.

In [ ]:
true_count = 0
false_count = 0
def evaluate_content_based_model():
    for key, colums in movies_df.iterrows():
        movies_recommended_by_model = get_recommendations_based_on_genres(colums["title"])
        predicted_genres  = get_movie_label(movies_recommended_by_model.index)
        for predicted_genre in predicted_genres:
            global true_count, false_count
            if predicted_genre == colums["genres"]:
                true_count = true_count+1
            else:
                false_count = false_count +1
evaluate_content_based_model()
total = true_count + false_count
print("Hit:"+ str(true_count/total))
print("Fault:" + str(false_count/total))

# Collaborative Filtering

## Item-Item Filtering

In [ ]:
df_movies_ratings=pd.merge(movies_df, rating_df)

In [ ]:
df_movies_ratings.head()

This code creates a pivot table `ratings_matrix_items` from the `df_movies_ratings` dataframe, where each row corresponds to a unique movieId, each column corresponds to a unique userId, and the values are the corresponding ratings. The resulting table is transposed and then any missing values (NaN) are filled with 0, resulting in a matrix of shape (number of movies, number of users).

In [ ]:
ratings_matrix_items = df_movies_ratings.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
ratings_matrix_items.fillna( 0, inplace = True )
ratings_matrix_items.shape

The `ratings_matrix_items` dataframe is a pivot table of the `df_movies_ratings` dataframe, with movie ids as the index, user ids as the columns, and rating values as the values. Any missing values are filled with 0. The resulting dataframe has 9724 rows (corresponding to unique movie ids) and 610 columns (corresponding to unique user ids).

In [ ]:
ratings_matrix_items.head()

In this code, the similarity between the movies is calculated based on their user ratings. The pairwise distances between the movies are calculated using the cosine similarity metric, and the diagonal is set to 0 as it represents the similarity of a movie with itself. The resulting similarity matrix is then converted into a pandas DataFrame for better handling.

In [ ]:
movie_similarity = 1 - pairwise_distances( ratings_matrix_items.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) 
ratings_matrix_items = pd.DataFrame( movie_similarity )
ratings_matrix_items.head()

This function takes in a movie name as input, and tries to find the movie in the `movies_df` dataframe. It then uses the precomputed `ratings_matrix_items` matrix to calculate the similarity between the input movie and all other movies in the dataframe. The similarity scores are stored in a new column called 'similarity' in the `movies_df` dataframe. If the input movie is not found in the dataframe, it prints a message saying "Sorry, the movie is not in the database!" but it doesn't return any value.

In [ ]:
def item_similarity(movieName): 
    try:
        user_inp=movieName
        inp=movies_df[movies_df['title']==user_inp].index.tolist()
        inp=inp[0]

        movies_df['similarity'] = ratings_matrix_items.iloc[inp]
        movies_df.columns = ['movie_id', 'title', 'release_date','similarity']
    except:
        print("Sorry, the movie is not in the database!")

This is a function that takes a user_id as input and returns a list of recommended movieIds based on item-item similarity of the movies previously rated by the user. 

The function starts by extracting movies rated with a 5 or 4.5 rating by the user. Then, it calls the item_similarity() function to find similar movies based on cosine similarity. The movies are sorted in descending order based on the similarity score, and movies with a similarity score greater than or equal to 0.45 are selected.

Next, the function creates a dataframe of recommended items that the user hasn't rated before, and selects the top 10 movies based on their rating. Finally, the function returns a list of the movieIds of these top 10 recommended movies.

In [ ]:
def recommendedMoviesAsperItemSimilarity(user_id):
    user_movie= df_movies_ratings[(df_movies_ratings.userId==user_id) & df_movies_ratings.rating.isin([5,4.5])][['title']]
    user_movie=user_movie.iloc[0,0]
    item_similarity(user_movie)
    sorted_movies_as_per_userChoice=movies_df.sort_values( ["similarity"], ascending = False )
    sorted_movies_as_per_userChoice=sorted_movies_as_per_userChoice[sorted_movies_as_per_userChoice['similarity'] >=0.45]['movie_id']
    recommended_movies=list()
    df_recommended_item=pd.DataFrame()
    user2Movies= rating_df[rating_df['userId']== user_id]['movieId']
    for movieId in sorted_movies_as_per_userChoice:
            if movieId not in user2Movies:
                df_new= rating_df[(rating_df.movieId==movieId)]
                df_recommended_item=pd.concat([df_recommended_item,df_new])
            best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:10] 
    return best10['movieId']

This function takes a list of movie IDs as input and returns a list of movie titles corresponding to the given IDs.

In [ ]:
def movieIdToTitle(listMovieIDs):
    movie_titles= list()
    for id in listMovieIDs:
        movie_titles.append(movies_df[movies_df['movie_id']==id]['title'])
    return movie_titles

The function recommendedMoviesAsperItemSimilarity takes a user ID as input and recommends 10 movies to that user based on item similarity. The function movieIdToTitle converts the list of recommended movie IDs to their corresponding titles.

In [ ]:
user_id=50
print("Recommended movies,:\n",movieIdToTitle(recommendedMoviesAsperItemSimilarity(user_id)))

# User-Item Filtering

code computes the user-user similarity matrix using the cosine similarity metric. It first creates a pivot table ratings_matrix_users from the df_movies_ratings dataframe with the rows as unique userId values, columns as unique movieId values, and the cells filled with the corresponding rating values. It then fills any missing values with zero using the fillna() method. Next, it computes the cosine similarity matrix between users using the pairwise_distances() function from the sklearn.metrics module and subtracting the result from 1. The diagonal elements of the similarity matrix are then set to zero using np.fill_diagonal() to avoid self-similarity. Finally, the similarity matrix is stored in a pandas dataframe called ratings_matrix_users.

In [ ]:
ratings_matrix_users = df_movies_ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating').reset_index(drop=True)
ratings_matrix_users.fillna( 0, inplace = True )
movie_similarity = 1 - pairwise_distances( ratings_matrix_users.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) 
ratings_matrix_users = pd.DataFrame( movie_similarity )
ratings_matrix_users.head()

The code `ratings_matrix_users.idxmax(axis=1)` returns the index of the movie (i.e. column) that has the highest similarity with each movie (i.e. row) in the user-item ratings matrix. In other words, it returns a Series object where each row represents a user and the value in that row is the index of the movie that is most similar to the movies that the user has rated. This can be used to make recommendations to users based on similar users' preferences.

In [ ]:
ratings_matrix_users.idxmax(axis=1)

This code selects 10 random rows from the `ratings_matrix_users` DataFrame and returns the index (i.e. userId) of the highest rated movie for each row. The `random_state` parameter is set to 10 to ensure that the same random rows are selected each time the code is run.

In [ ]:
ratings_matrix_users.idxmax(axis=1).sample( 10, random_state = 10 )

The code creates a Pandas DataFrame (`df_similar_user`) from a Pandas Series (`similar_user_series`) that contains the user ID of the most similar user for each user in the ratings matrix (`ratings_matrix_users`). The `idxmax()` method is used to find the index (i.e., the user ID) of the maximum value in each row of the ratings matrix, which corresponds to the most similar user for that user. The resulting Pandas Series is then converted to a DataFrame using the `to_frame()` method.

In [ ]:
similar_user_series= ratings_matrix_users.idxmax(axis=1)
df_similar_user= similar_user_series.to_frame()

This code assigns the name 'similarUser' to the only column in the DataFrame `df_similar_user`. Specifically, it does this by setting the DataFrame column names attribute `df_similar_user.columns` to a list containing a single string element, which is 'similarUser'.

In [ ]:
df_similar_user.columns=['similarUser']

The `df_similar_user` DataFrame has one column named `similarUser`, which contains the user ID of the most similar user for each user in the `ratings_matrix_users` DataFrame.

In [ ]:
df_similar_user.head()

The function `getRecommendedMoviesAsperUserSimilarity` takes a `userId` as input and returns a list of recommended movies based on user similarity. It first extracts all the movies watched by the user and then finds the most similar user based on their movie preferences. Using this similar user, it then recommends movies that the similar user has watched but the original user has not watched yet. The function returns a list of recommended movie IDs.

In [ ]:
movieId_recommended=list()
def getRecommendedMoviesAsperUserSimilarity(userId):
    user2Movies= rating_df[rating_df['userId']== userId]['movieId']
    sim_user=df_similar_user.iloc[0,0]
    df_recommended=pd.DataFrame(columns=['movieId','title','genres','userId','rating','timestamp'])
    for movieId in rating_df[rating_df['userId']== sim_user]['movieId']:
        if movieId not in user2Movies:
            df_new= df_movies_ratings[(df_movies_ratings.userId==sim_user) & (df_movies_ratings.movieId==movieId)]
            df_recommended=pd.concat([df_recommended,df_new])
        best10=df_recommended.sort_values(['rating'], ascending = False )[1:10]  
    return best10['movieId']

 code provided performs movie recommendation using two methods - item similarity and user similarity. The item similarity approach recommends movies based on the similarity between items (movies) themselves, while the user similarity approach recommends movies based on the similarity between users' preferences. The code uses collaborative filtering to identify similar users and movies, and then recommends movies that are highly rated by similar users or similar to highly-rated movies by the user. The code makes use of Pandas and NumPy libraries for data manipulation, and the scikit-learn library for pairwise distance calculation.

In [ ]:
user_id=50
recommend_movies= movieIdToTitle(getRecommendedMoviesAsperUserSimilarity(user_id))
print("Movies you should watch are:\n")
print(recommend_movies)

function takes in two user IDs as input and returns the list of movies that both users have rated. It does so by merging the dataframes of both users, keeping only the movies that appear in both dataframes. The function drops unnecessary columns and returns the resulting dataframe containing the user IDs, movie titles, and ratings.

In [ ]:
def get_user_similar_movies( user1, user2 ):
    common_movies = df_movies_ratings[df_movies_ratings.userId == user1].merge(
      df_movies_ratings[df_movies_ratings.userId == user2],
      on = "movieId",
      how = "inner" )
    common_movies.drop(['movieId','genres_x','genres_y', 'timestamp_x','timestamp_y','title_y'],axis=1,inplace=True)
    return common_movies

In [ ]:
get_user_similar_movies(587,511)